<a href="https://colab.research.google.com/github/piyasharma-g/repo-p/blob/main/go_agentspace_agents_vertexai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# go/agentspace-agents-vertexai
* Deploy Agentspace agents using Vertex AI
* Continuously tested and updated with public [ADK](https://github.com/google/adk-python)
* For issues or suggestions, contact @artw
* Special thanks to feedback from @krzysiekw, @jamag, @ykoh, @ajiteshk

# 1. Create an Agentspace app
If you don't have an Agentspace app, follow instructions here:
https://docs.google.com/document/d/1Mkmi7cCXXavUB6ZQYIAp1CIE7mv7Dl0PyN0OZWUufvI

# 2. Install ADK, Agent Engines, and your other required libraries

In [ ]:
!pip3 install -qq -U google-adk deprecated "google-cloud-aiplatform[agent_engines]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

## 2b. Restart notebook after installing libraries above

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

# 3. Authenticate
We'll use `ucs-ga-fishfood-1` in this notebook as an example but be sure to substitute it with your own project.

In [ ]:
import os

PROJECT_ID = "ucs-ga-fishfood-1"  # @param {type:"string"}
LOCATION = "us-central1" # @param {type:"string"}
GCS_BUCKET = "gs://agent-deployment-pantheon" # @param {type:"string"}

from google.colab import auth
auth.authenticate_user(project_id=PROJECT_ID)

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

## 3b. Ensure necessary libraries are imported

In [ ]:
from google.adk import Agent

import vertexai
from vertexai import agent_engines

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=GCS_BUCKET,
)

# 4. Define agent

In [ ]:
def roll_die(sides: int) -> int:
 """Roll a die and return the rolled result.


 Args:
   sides: The integer number of sides the die has.


 Returns:
   An integer of the result of rolling the die.
 """
 import random


 return random.randint(1, sides)


def check_prime(nums: list[int]) -> list[str]:
 """Check if a given list of numbers are prime.


 Args:
   nums: The list of numbers to check.


 Returns:
   A str indicating which number is prime.
 """


 primes = set()
 for number in nums:
   number = int(number)
   if number <= 1:
     continue
   is_prime = True
   for i in range(2, int(number**0.5) + 1):
     if number % i == 0:
       is_prime = False
       break
   if is_prime:
     primes.add(number)
 return (
     'No prime numbers found.'
     if not primes
     else f"{', '.join(str(num) for num in primes)} are prime numbers."
 )


root_agent = Agent(
   model="gemini-2.0-flash-001",
   name='dice_rolling_agent',
   instruction="""
     You roll dice and answer questions about the outcome of the dice rolls.
     You can roll dice of different sizes.
     You can use multiple tools in parallel by calling functions in parallel(in one request and in one round).
     The only things you do are roll dice for the user and discuss the outcomes.
     It is ok to discuss previous dice roles, and comment on the dice rolls.
     When you are asked to roll a die, you must call the roll_die tool with the number of sides. Be sure to pass in an integer. Do not pass in a string.
     You should never roll a die on your own.
     When checking prime numbers, call the check_prime tool with a list of integers. Be sure to pass in a list of integers. You should never pass in a string.
     You should not check prime numbers before calling the tool.
     When you are asked to roll a die and check prime numbers, you should always make the following two function calls:
     1. You should first call the roll_die tool to get a roll. Wait for the function response before calling the check_prime tool.
     2. After you get the function response from roll_die tool, you should call the check_prime tool with the roll_die result.
       2.1 If user asks you to check primes based on previous rolls, make sure you include the previous rolls in the list.
     3. When you respond, you must include the roll_die result from step 1.
     You should always perform the previous 3 steps when asking for a roll and checking prime numbers.
     You should not rely on the previous history on prime results.
   """,
   tools=[roll_die, check_prime],
)

# 5. Define wrapper app

In [ ]:
from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

## Test locally

In [ ]:
session = app.create_session(user_id="u_123")
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="roll a 6-sided die",
):
  print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-0d668c39-c577-4b70-9735-5118708445ea', 'args': {'sides': 6}, 'name': 'roll_die'}}], 'role': 'model'}, 'invocation_id': 'e-0d780843-1d11-4b91-a8a2-a3aafc18f60d', 'author': 'dice_rolling_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'ZpkX1Z0h', 'timestamp': 1747916413.959229}
{'content': {'parts': [{'function_response': {'id': 'adk-0d668c39-c577-4b70-9735-5118708445ea', 'name': 'roll_die', 'response': {'result': 5}}}], 'role': 'user'}, 'invocation_id': 'e-0d780843-1d11-4b91-a8a2-a3aafc18f60d', 'author': 'dice_rolling_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'C8Tx0bM5', 'timestamp': 1747916415.783834}
{'content': {'parts': [{'text': 'I rolled a 6-sided die and got a 5.\n'}], 'role': 'model'}, 'invocation_id': 'e-0d780843-1d11-4b91-a8a2-a3aafc18f60d', 'author': 'dice_rolling_agent', 'actions': {'sta

In [ ]:
agent_context = '{"message":{"role":"user","parts":[{"text":"How were you built?"}]},"events":[{"content":{"role":"user","parts":[{"text":"how were you built ?"}]},"author":"AgentSpace_root_agent"},{"content":{"role":"model","parts":[{"functionCall":{"name":"agentspaceak","args":{"question":"How were you built?"},"id":"14076651604820872102"}}]},"invocation_id":"14076651604820871801","author":"AgentSpace_root_agent","id":"14076651604820872102"}]}'
for response in app.streaming_agent_run_with_events(agent_context):
    for event in response["events"]:
      print(event)

# 6. Deploy to GCP

In [ ]:
from vertexai import agent_engines

deployed_engine = agent_engines.create(
    agent_engine=app,
    display_name="DICE_AGENT",
    requirements=[
        "google-adk",
        "google-cloud-aiplatform[agent_engines]",
        "deprecated"
    ],
    extra_packages=[
    ],
)

# 7. Add Authorization (if you need it)
7a. Add an OAuth 2.0 client with proper scopes and redirect URL:
https://vertexaisearch.cloud.google.com/oauth-redirect

Example below is based on [Google's OAuth 2.0 client](https://support.google.com/googleapi/answer/6158849):
- From the GCP console, search for "OAuth consent screen"
- Create an OAuth 2.0 client for Web application
- Add the redirect URL specified above
- Click **Create**
- Download the `client_secret.json` and upload it here
- Run the Python script below to obtain the authorization URI

For other 3rd party OAuth clients, follow their documentation to obtain authorization information.

In [ ]:
import google.oauth2.credentials
import google_auth_oauthlib.flow

# Required, call the from_client_secrets_file method to retrieve the client ID from a
# client_secret.json file. The client ID (from that file) and access scopes are required. (You can
# also use the from_client_config method, which passes the client configuration as it originally
# appeared in a client secrets file but doesn't access the file itself.)
flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file('client_secret.json',
    scopes=['https://www.googleapis.com/auth/drive.metadata.readonly',
            'https://www.googleapis.com/auth/calendar.readonly'])

# Required, indicate where the API server will redirect the user after the user completes
# the authorization flow. The redirect URI is required. The value must exactly
# match one of the authorized redirect URIs for the OAuth 2.0 client, which you
# configured in the API Console. If this value doesn't match an authorized URI,
# you will get a 'redirect_uri_mismatch' error.
flow.redirect_uri = 'https://vertexaisearch.cloud.google.com/oauth-redirect'

# Generate URL for request to Google's OAuth 2.0 server.
# Use kwargs to set optional request parameters.
authorization_url, state = flow.authorization_url(
    # Recommended, enable offline access so that you can refresh an access token without
    # re-prompting the user for permission. Recommended for web server apps.
    access_type='offline',
    # Optional, enable incremental authorization. Recommended as a best practice.
    include_granted_scopes='true',
    # Optional, if your application knows which user is trying to authenticate, it can use this
    # parameter to provide a hint to the Google Authentication Server.
    login_hint='hint@example.com',
    # Optional, set prompt to 'consent' will prompt the user for consent
    prompt='consent')
print('OAUTH_AUTH_URI:')
print(authorization_url)

OAUTH_AUTH_URI:
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=862721868538-ep7fj2ldbnqcto289rmmbnfe11atdq73.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fvertexaisearch.cloud.google.com%2Foauth-redirect&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=kh7gAm5Z815uMj7FBNZXIQCCNAqTYV&access_type=offline&include_granted_scopes=true&login_hint=hint%40example.com&prompt=consent


7b. Then link the authorization to Agentspace below.
- Make something up for `AUTH_ID` and save it to reference later.
- Enter the `OAUTH_CLIENT_ID` and `OAUTH_CLIENT_SECRET` from the OAuth client setup in the previous steps.
- Ensure that `OAUTH_TOKEN_URI` is what your service requires. Example used here is for Google APIs.

In [ ]:
%%bash
export PROJECT_NUMBER="862721868538"
export AUTH_ID="madeup-id-123"
export OAUTH_CLIENT_ID="XXXXXX"
export OAUTH_CLIENT_SECRET="XXXXXX"
export OAUTH_AUTH_URI="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=862721868538-ep7fj2ldbnqcto289rmmbnfe11atdq73.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fvertexaisearch.cloud.google.com%2Foauth-redirect&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=kh7gAm5Z815uMj7FBNZXIQCCNAqTYV&access_type=offline&include_granted_scopes=true&login_hint=hint%40example.com&prompt=consent"
export OAUTH_TOKEN_URI="https://oauth2.googleapis.com/token"

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  -H "X-Goog-User-Project: ${PROJECT_NUMBER}" \
https://discoveryengine.googleapis.com/v1alpha/projects/${PROJECT_NUMBER}/locations/global/authorizations?authorizationId=${AUTH_ID} \
  -d '{
  "name": "projects/'"${PROJECT_NUMBER}"'/locations/global/authorizations/'"${AUTH_ID}"'",
  "serverSideOauth2": {
      "clientId": "'"${OAUTH_CLIENT_ID}"'",
      "clientSecret": "'"${OAUTH_CLIENT_SECRET}"'",
      "authorizationUri": "'"${OAUTH_AUTH_URI}"'",
      "tokenUri": "'"${OAUTH_TOKEN_URI}"'"
    }
  }'

{
  "name": "projects/862721868538/locations/global/authorizations/madeup-id-123",
  "serverSideOauth2": {
    "clientId": "XXXXXX",
    "clientSecret": "XXXXXX",
    "tokenUri": "https://oauth2.googleapis.com/token",
    "authorizationUri": "https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=862721868538-ep7fj2ldbnqcto289rmmbnfe11atdq73.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fvertexaisearch.cloud.google.com%2Foauth-redirect&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=kh7gAm5Z815uMj7FBNZXIQCCNAqTYV&access_type=offline&include_granted_scopes=true&login_hint=hint%40example.com&prompt=consent"
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1475    0   732  100   743   1915   1944 --:--:-- --:--:-- --:--:--  3871


# 8. Update config to link Agent to Agentspace

In [ ]:
%%bash
export PROJECT_NUMBER="862721868538"
export REASONING_ENGINE="projects/862721868538/locations/us-central1/reasoningEngines/6856317016299536384"
export DISPLAY_NAME="Dice Agent"
export DESCRIPTION="Roll dice and track results."
export TOOL_DESCRIPTION="The agent rolls dice and tracks the roll results."
export AGENT_ID="dice_agent"
export AS_APP="itsm-agent-dogfood_1742417292379"
export AUTH_ID="madeup-id-123"

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  -H "X-Goog-User-Project: ${PROJECT_NUMBER}" \
https://discoveryengine.googleapis.com/v1alpha/projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant/agents \
  -d '{
      "displayName": "'"${DISPLAY_NAME}"'",
      "description": "'"${DESCRIPTION}"'",
      "adk_agent_definition": {
        "tool_settings": {
          "tool_description": "'"${TOOL_DESCRIPTION}"'"
        },
        "provisioned_reasoning_engine": {
          "reasoning_engine":
            "'"${REASONING_ENGINE}"'"
        },
        "authorizations": [
          "projects/'"${PROJECT_NUMBER}"'/locations/global/authorizations/'"${AUTH_ID}"'"
        ]
      }
  }'

{
  "name": "projects/862721868538/locations/global/collections/default_collection/engines/itsm-agent-dogfood_1742417292379/assistants/default_assistant/agents/1107892644173037518",
  "displayName": "Dice Agent",
  "description": "Roll dice and track results.",
  "adkAgentDefinition": {
    "toolSettings": {
      "toolDescription": "The agent rolls dice and tracks the roll results."
    },
    "provisionedReasoningEngine": {
      "reasoningEngine": "projects/862721868538/locations/us-central1/reasoningEngines/6856317016299536384"
    },
    "authorizations": [
      "projects/862721868538/locations/global/authorizations/madeup-id-123"
    ]
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1206    0   657  100   549   3389   2832 --:--:-- --:--:-- --:--:--  6248


# 9. Try new Agentspace UI
1. Ensure that `Vertex AI API` and `Discovery Engine API` are enabled in your project and that Discovery Engine service account has `Vertex AI User` permission. If you cannot find it in IAM, check the `Include Google-provided role grants`.
2. Open up the Agentspace app in GCP console
3. From left menu, click `Integration`
4. Open the URL provided
5. From left menu, select the agent deployed

#10. More Operations
To view, list, update or delete your configurations, see https://drive.google.com/file/d/1BSbt_UUNdzy5YvEVuDez7RihMdJ6pbul/view?resourcekey=0-IYbPfqERhbtRzSu_gvHy6A

In [ ]:
%%bash
export PROJECT_NUMBER="862721868538"
export AUTH_ID="madeup-id-123"

curl -X DELETE \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  -H "X-Goog-User-Project: ${PROJECT_NUMBER}" \
https://discoveryengine.googleapis.com/v1alpha/projects/${PROJECT_NUMBER}/locations/global/authorizations/${AUTH_ID}

{}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     3    0     3    0     0      6      0 --:--:-- --:--:-- --:--:--     6
